In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from gmf import GMFEngine
from mlp import MLPEngine
from neumf import NeuMFEngine
from data import SampleGenerator
import time

In [2]:
def to_timestamp(date_string):
    return time.mktime(time.strptime(date_string, '%Y-%m-%dT%H:%M:%S.000Z'))

In [ ]:
snomed_codes = pd.read_csv('diagnosis_big.csv', usecols = ['snomed_id'])
frequent_snomed_codes = snomed_codes.value_counts()[100:1100].index

diagnosis_df = pd.read_csv('diagnosis_big.csv', usecols = ['explorys_patient_id','snomed_id', 'diagnosis_date'])
diagnosis_df = diagnosis_df[np.isin(diagnosis_df.snomed_id, frequent_snomed_codes)]
frequent_users = diagnosis_df.explorys_patient_id.value_counts()[:-200000].index
diagnosis_df = diagnosis_df[np.isin(diagnosis_df.explorys_patient_id, frequent_users)]

user_id = diagnosis_df[['explorys_patient_id']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
diagnosis_df = pd.merge(diagnosis_df, user_id, on=['explorys_patient_id'], how='left')
item_id = diagnosis_df[['snomed_id']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
diagnosis_df = pd.merge(diagnosis_df, item_id, on=['snomed_id'], how='left')

diagnosis_df['timestamp'] = diagnosis_df.diagnosis_date.apply(lambda x: to_timestamp(x))

diagnosis_df['rating'] = 1
diagnosis_df.to_csv('diagnosis_data.csv', index=False)
diagnosis_df = diagnosis_df[['userId', 'itemId', 'rating', 'timestamp']]

print('Range of userId is [{}, {}]'.format(diagnosis_df.userId.min(), diagnosis_df.userId.max()))
print('Range of itemId is [{}, {}]'.format(diagnosis_df.itemId.min(), diagnosis_df.itemId.max()))

In [4]:
# Check sparsity
len(diagnosis_df) / (diagnosis_df.itemId.max()*diagnosis_df.userId.max())

0.06597158096326317

In [5]:
sample_generator = SampleGenerator(ratings=diagnosis_df)
evaluate_data = sample_generator.evaluate_data

/Users/xu.b.zheng/neural-collaborative-filtering/src/data.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['rating'][ratings['rating'] > 0] = 1.0
